<a href="https://colab.research.google.com/github/16A0/til/blob/master/Text2Image_FFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image

Based on [CLIP](https://github.com/openai/CLIP) + FFT from [Lucent](https://github.com/greentfrapp/lucent) // made by [eps696](https://github.com/eps696)

## Features 
* uses image and/or text as prompts
* generates [FFT-encoded](https://github.com/greentfrapp/lucent/blob/master/lucent/optvis/param/spatial.py) image (detailed tiled textures, a la deepdream)
* ! very fast convergence
* ! undemanding for RAM (fullHD resolution and more)


**Run this cell after each session restart**

In [ ]:
#@title General setup

import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  # from pprint import pprint
  translator = Translator()
except: pass
!pip install ftfy

import os
import time
import random
import imageio
import numpy as np
import PIL
from skimage import exposure
from base64 import b64encode

import torch
import torch.nn as nn
import torchvision

from IPython.display import HTML, Image, display, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import ipywidgets as ipy
# import glob
from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!git clone https://github.com/openai/CLIP.git
%cd /content/CLIP/
import clip
perceptor, preprocess = clip.load('ViT-B/32')

workdir = '_out'
tempdir = os.path.join(workdir, 'ttt')
os.makedirs(tempdir, exist_ok=True)

clear_output()

###  FFT from Lucent library  https://github.com/greentfrapp/lucent

def pixel_image(shape, sd=2.):
    tensor = (torch.randn(*shape) * sd).cuda().requires_grad_(True)
    return [tensor], lambda: tensor

# From https://github.com/tensorflow/lucid/blob/master/lucid/optvis/param/spatial.py
def rfft2d_freqs(h, w):
    """Computes 2D spectrum frequencies."""
    fy = np.fft.fftfreq(h)[:, None]
    # when we have an odd input dimension we need to keep one additional frequency and later cut off 1 pixel
    if w % 2 == 1:
        fx = np.fft.fftfreq(w)[: w // 2 + 2]
    else:
        fx = np.fft.fftfreq(w)[: w // 2 + 1]
    return np.sqrt(fx * fx + fy * fy)

def fft_image(shape, sd=0.1, decay_power=1., smooth_col=1.):
    batch, channels, h, w = shape
    freqs = rfft2d_freqs(h, w)
    init_val_size = (batch, channels) + freqs.shape + (2,) # 2 for imaginary and real components
    spectrum_real_imag_t = (torch.randn(*init_val_size) * sd).cuda().requires_grad_(True)
    scale = 1.0 / np.maximum(freqs, 1.0 / max(w, h)) ** decay_power
    scale = torch.tensor(scale).float()[None, None, ..., None].cuda()

    def inner():
        scaled_spectrum_t = scale * spectrum_real_imag_t
        image = torch.irfft(scaled_spectrum_t, 2, normalized=True, signal_sizes=(h, w))
        image = image[:batch, :channels, :h, :w]
        image = image / smooth_col # reduce saturation, smoothen colors & contrast
        return image
    return [spectrum_real_imag_t], inner

def to_valid_rgb(image_f, decorrelate=True):
    def inner():
        image = image_f()
        if decorrelate:
            image = _linear_decorrelate_color(image)
        return torch.sigmoid(image)
    return inner
    
def _linear_decorrelate_color(tensor):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    t_permute = tensor.permute(0,2,3,1)
    t_permute = torch.matmul(t_permute, torch.tensor(color_correlation_normalized.T).to(device))
    tensor = t_permute.permute(0,3,1,2)
    return tensor

color_correlation_svd_sqrt = np.asarray([[0.26, 0.09, 0.02],
                                         [0.27, 0.00, -0.05],
                                         [0.27, -0.09, 0.03]]).astype("float32")
max_norm_svd_sqrt = np.max(np.linalg.norm(color_correlation_svd_sqrt, axis=0))
color_correlation_normalized = color_correlation_svd_sqrt / max_norm_svd_sqrt

### Libs

def slice_imgs(imgs, count, transform=None, uniform=True):
  def map(x, a, b):
    return x * (b-a) + a
  rnd_size = torch.rand(count)
  if uniform is True:
    rnd_offx = torch.rand(count)
    rnd_offy = torch.rand(count)
  else: # ~normal around center
    rnd_offx = torch.clip(torch.randn(count) * 0.2 + 0.5, 0, 1) 
    rnd_offy = torch.clip(torch.randn(count) * 0.2 + 0.5, 0, 1)
  
  sz = [img.shape[2:] for img in imgs]
  sz_min = [np.min(s) for s in sz]
  if uniform is True:
    sz = [[2*s[0], 2*s[1]] for s in list(sz)]
    imgs = [pad_up_to(imgs[i], sz[i], type='centr') for i in range(len(imgs))]

  sliced = []
  for i, img in enumerate(imgs):
    cuts = []
    for c in range(count):
      csize = map(rnd_size[c], 224, 0.98*sz_min[i]).int()
      offsetx = map(rnd_offx[c], 0, sz[i][1] - csize).int()
      offsety = map(rnd_offy[c], 0, sz[i][0] - csize).int()
      cut = img[:, :, offsety:offsety + csize, offsetx:offsetx + csize]
      cut = torch.nn.functional.interpolate(cut, (224,224), mode='bilinear')
      if transform is not None: 
          cut = transform(cut)
      cuts.append(cut)
    sliced.append(torch.cat(cuts, 0))
  return sliced

def makevid(seq_dir, size=None):
  out_sequence = seq_dir + '/%03d.jpg'
  out_video = seq_dir + '.mp4'
  !ffmpeg -y -v warning -i $out_sequence $out_video
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

# Tiles an array around two points, allowing for pad lengths greater than the input length
# adapted from https://discuss.pytorch.org/t/symmetric-padding/19866/3
def tile_pad(xt, padding):
  h, w = xt.shape[-2:]
  left, right, top, bottom = padding

  def tile(x, minx, maxx):
    rng = maxx - minx
    mod = np.remainder(x - minx, rng)
    out = mod + minx
    return np.array(out, dtype=x.dtype)

  x_idx = np.arange(-left, w+right)
  y_idx = np.arange(-top, h+bottom)
  x_pad = tile(x_idx, -0.5, w-0.5)
  y_pad = tile(y_idx, -0.5, h-0.5)
  xx, yy = np.meshgrid(x_pad, y_pad)
  return xt[..., yy, xx]

def pad_up_to(x, size, type='centr'):
  sh = x.shape[2:][::-1]
  if list(x.shape[2:]) == list(size): return x
  padding = []
  for i, s in enumerate(size[::-1]):
    if 'side' in type.lower():
      padding = padding + [0, s-sh[i]]
    else: # centr
      p0 = (s-sh[i]) // 2
      p1 = s-sh[i] - p0
      padding = padding + [p0,p1]
  y = tile_pad(x, padding)
  return y

class ProgressBar(object):
  def __init__(self, task_num=10):
    self.pbar = ipy.IntProgress(min=0, max=task_num, bar_style='') # (value=0, min=0, max=max, step=1, description=description, bar_style='')
    self.labl = ipy.Label()
    display(ipy.HBox([self.pbar, self.labl]))
    self.task_num = task_num
    self.completed = 0
    self.start()

  def start(self, task_num=None):
    if task_num is not None:
      self.task_num = task_num
    if self.task_num > 0:
      self.labl.value = '0/{}'.format(self.task_num)
    else:
      self.labl.value = 'completed: 0, elapsed: 0s'
    self.start_time = time.time()

  def upd(self, *p, **kw):
    self.completed += 1
    elapsed = time.time() - self.start_time + 0.0000000000001
    fps = self.completed / elapsed if elapsed>0 else 0
    if self.task_num > 0:
      finaltime = time.asctime(time.localtime(self.start_time + self.task_num * elapsed / float(self.completed)))
      fin = ' end %s' % finaltime[11:16]
      percentage = self.completed / float(self.task_num)
      eta = int(elapsed * (1 - percentage) / percentage + 0.5)
      self.labl.value = '{}/{}, rate {:.3g}s, time {}s, left {}s, {}'.format(self.completed, self.task_num, 1./fps, shortime(elapsed), shortime(eta), fin)
    else:
      self.labl.value = 'completed {}, time {}s, {:.1f} steps/s'.format(self.completed, int(elapsed + 0.5), fps)
    self.pbar.value += 1
    if self.completed == self.task_num: self.pbar.bar_style = 'success'
    return 
    # return self.completed

def time_days(sec):
  return '%dd %d:%02d:%02d' % (sec/86400, (sec/3600)%24, (sec/60)%60, sec%60)
def time_hrs(sec):
  return '%d:%02d:%02d' % (sec/3600, (sec/60)%60, sec%60)
def shortime(sec):
  if sec < 60:
    time_short = '%d' % (sec)
  elif sec < 3600:
    time_short  = '%d:%02d' % ((sec/60)%60, sec%60)
  elif sec < 86400:
    time_short  = time_hrs(sec)
  else:
    time_short = time_days(sec)
  return time_short


!nvidia-smi -L
print('\nDone!')

Type some text to hallucinate it, or upload some image to neuremix it.  
Or use both, why not.

In [ ]:
#@title Input

text = "" #@param {type:"string"}
translate = False #@param {type:"boolean"}
#@markdown or 
upload_image = False #@param {type:"boolean"}

if translate:
  text = translator.translate(text, dest='en').text
if upload_image:
  uploaded = files.upload()

This method converges pretty fast, yet you may set more iterations just to get that nice animation (the result may get better as well).  
`smooth_col` scaler desaturates image, while uncovering more details (the bigger the smoother).

In [ ]:
#@title Generate

# from google.colab import drive
# drive.mount('/content/GDrive')
# clipsDir = '/content/GDrive/MyDrive/T2I ' + dtNow.strftime("%Y-%m-%d %H%M")

!rm -rf tempdir

sideX = 1280 #@param {type:"integer"}
sideY = 720 #@param {type:"integer"}
smooth_col =  2.#@param {type:"number"}
uniform = True #@param {type:"boolean"}
#@markdown > Training
steps = 100 #@param {type:"integer"}
samples = 100 #@param {type:"integer"}
learning_rate = .05 #@param {type:"number"}
#@markdown > Misc
save_freq = 1 #@param {type:"integer"}
audio_notification = False #@param {type:"boolean"}

norm_in = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

if upload_image:
  input = list(uploaded.values())[0]
  print(list(uploaded)[0])
  img_in = torch.from_numpy(imageio.imread(input).astype(np.float32)/255.).unsqueeze(0).permute(0,3,1,2).cuda()
  in_sliced = slice_imgs([img_in], samples, transform=norm_in)[0]
  img_enc = perceptor.encode_image(in_sliced).detach().clone()
  del img_in, in_sliced; torch.cuda.empty_cache()

if len(text) > 2:
  print(text)
  if translate:
    translator = Translator()
    text = translator.translate(text, dest='en').text
    print(' translated to:', text) 
  tx = clip.tokenize(text)
  txt_enc = perceptor.encode_text(tx.cuda()).detach().clone()

shape = [1, 3, sideY, sideX]
param_f = fft_image 
# param_f = pixel_image
# learning_rate = 1.
params, image_f = param_f(shape, smooth_col=smooth_col)
image_f = to_valid_rgb(image_f)
optimizer = torch.optim.Adam(params, learning_rate)

def displ(img, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1,2,0))  
  img = exposure.equalize_adapthist(np.clip(img, -1., 1.))
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave('result.jpg', np.array(img))

def checkin(num):
  with torch.no_grad():
    img = image_f().cpu().numpy()[0]
  displ(img, os.path.join(tempdir, '%03d.jpg' % num))
  outpic.clear_output()
  with outpic:
    display(Image('result.jpg'))

def train(i):
  loss = 0
  img_out = image_f()
  imgs_sliced = slice_imgs([img_out], samples, norm_in, uniform=uniform)
  out_enc = perceptor.encode_image(imgs_sliced[-1])
  loss = 0
  if upload_image:
    loss += -100*torch.cosine_similarity(img_enc, out_enc, dim=-1).mean()
  if len(text) > 2:
    loss += -100*torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
  if isinstance(loss, int): print(' Loss not defined, check the inputs'); exit(1)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
  if i % save_freq == 0:
    checkin(i // save_freq)

outpic = ipy.Output()
outpic

pbar = ProgressBar(steps)
for i in range(steps):
  train(i)
  _ = pbar.upd()

HTML(makevid(tempdir))
files.download('_out/ttt.mp4')
if audio_notification == True: output.eval_js('new Audio("https://freesound.org/data/previews/80/80921_1022651-lq.ogg").play()')
